In [17]:
import torch
from transformers import AutoTokenizer, AutoConfig

import numpy as np
import pandas as pd

from TRC.models import ModelForWeightedSequenceClassification
from common_utils import extract_from_dataframe

In [12]:
ASSIGN_WEIGHT = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = 'roberta-large'

model_path = '/home/cc/rora_tesi_new/log/log_TRC/roberta-large/bertweet-seq/20_epoch/data/True_weight/42_seed/saved-model/pytorch_model.bin'
config_path = '/home/cc/rora_tesi_new/log/log_TRC/roberta-large/bertweet-seq/20_epoch/data/True_weight/42_seed/saved-model/config.json'

In [16]:
def load_local_model(model_path, config_path, device, model_name):

    config = config = AutoConfig.from_pretrained(config_path)

    model = ModelForWeightedSequenceClassification(model_name=model_name,config=config)
    checkpoint = torch.load(model_path, map_location=device)
    model.load_state_dict(checkpoint)

    return model

def create_weight(Y_train):
    # weight of each class in loss function
    class_weight = [np.array(Y_train).shape[0] / (np.array(Y_train) == i).sum() for i in range(len(set(Y_train)))]
    class_weight = torch.FloatTensor(class_weight)
    return class_weight

def prepare_data(data_path, need_columns):

    data = pd.read_pickle(data_path)
    X, Y = extract_from_dataframe(data, need_columns)
    return X, Y

In [ ]:
train_data_path = '/home/cc/rora_tesi_new/data/train.p'
test_data_path = '/home/cc/rora_tesi_new/data/test.p'

need_columns = ['tweet_tokens', 'sentence_class']

_, Y_train = prepare_data(train_data_path, need_columns)
X_test_raw, Y_test = prepare_data(test_data_path, need_columns)

test_batch_size = Y_test.shape[0]


In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_name, normalization = True)
model = load_local_model(model_path, config_path, device, model_name)
labels = sorted(model.config.label2id, key=model.config.label2id.get)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class_weight = None
if ASSIGN_WEIGHT:
    class_weight = create_weight(Y_train)

In [15]:
def f(model, test_batch_generator, num_batches, device, class_weight):
    pass

['LABEL_0', 'LABEL_1']
